In [ ]:
!pip install pytesseract pdfplumber pillow langchain pydantic

### WINDOWS
Please install tesseract from the following link: https://github.com/UB-Mannheim/tesseract/wiki

### LINUX
```sudo apt-get update```

```sudo apt-get install python3-pil tesseract-ocr libtesseract-dev tesseract-ocr-eng tesseract-ocr-script-latn```

In [7]:
import pytesseract
import pdfplumber
from PIL import Image
import io, re


In [ ]:
        ## IF YOU'RE ON WINDOWS, EXECUTE THIS CELL ##
        ## IF YOU'RE ON WINDOWS, EXECUTE THIS CELL ##
        ## IF YOU'RE ON WINDOWS, EXECUTE THIS CELL ##



pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [37]:
## ENTER YOUR OPENAI_API_KEY HERE

OPENAI_API_KEY=''

In [27]:
def preprocess_text(text):
    """Clean and preprocess extracted text."""
    text = re.sub(r"\s+", " ", text)  # Remove extra spaces
    return text.strip()


def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF pitch deck using OCR for all pages."""
    text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
        
            # Convert the page to an image and apply OCR
            img = page.to_image(resolution=300).original
            img_bytes = io.BytesIO()
            img.save(img_bytes, format="PNG")
            img = Image.open(img_bytes)
            
            ocr_text = pytesseract.image_to_string(img)
            text.append(ocr_text)

    return preprocess_text("\n".join(text))


In [14]:
from pydantic import BaseModel, Field

class StartupPitch(BaseModel):
    """The data on a startup pitch"""
    problem_score: str = Field(description='The score assigned to the problem statement section as a startup pitch [0-100], followed by strengths, weaknesses and suggestions to improve it.')
    solution_score: str = Field(description='The score assigned to the solution section as a startup pitch [0-100], followed by strengths, weaknesses and suggestions to improve it.') 
    market_score: str = Field(description='The score assigned to the market section as a startup pitch [0-100], followed by strengths, weaknesses and suggestions to improve it.')
    business_model_score: str = Field(description='The score assigned to the business model section as a startup pitch [0-100], followed by strengths, weaknesses and suggestions to improve it.')
    financials_score: str = Field(description='The score assigned to the financials section as a startup pitch [0-100], followed by strengths, weaknesses and suggestions to improve it.')
    team_score: str = Field(description='The score assigned to the team section as a startup pitch [0-100], followed by strengths, weaknesses and suggestions to improve it.')
    def __str__(self):
        s = f'Problem Statement Analysis:\n{self.problem_score}\n\nSolution Analysis:\n{self.solution_score}\n\nMarket Analysis:\n{self.market_score}\n\nBusiness Model Analysis:\n{self.business_model_score}\n\nFinancials Analysis:\n{self.financials_score}\n\nTeam Analysis:\n{self.team_score}\n\n'
        return s

In [36]:
pitch = extract_text_from_pdf('data/pitch-decks/uber-pitch-deck.pdf')

In [ ]:
pitch

In [34]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai",api_key=OPENAI_API_KEY).with_structured_output(StartupPitch)




cls = llm.invoke(pitch)


In [ ]:
print(cls)